# Agent with Long-Time Memory
* We will build an Agent that will help us to **manage a ToDo list**.
* It will decide:
    * **when to save items** to our ToDo list.
    * **to save either a user profile or a collection of ToDo items**.
* In addition to semantic memory (user facts), it will also have **procedural memory**.
    * Remember, the procedural memory is the system prompt. This will allow the user to set preferences for creating ToDo items.

In [ ]:
#pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ.get("Open_ai_key_here")

Lets install Langchain here...



In [ ]:
#!pip install langchain-openai

In [ ]:
from langchain_openai import ChatOpenA

chatModel35 = ChatOpenAI(model="gpt-3.5-turbo-0125")
chatModel4o = ChatOpenAI(model = "gpt-4o")



Using TrustCall here....

In [ ]:
from pydantic import BaseModel, Field

class Memory(BaseModel):
    conent: str = Field(description= "The main content of the memory. For example:" \
    "User expressed interest in learning about French language...")

class MemoryCollection(BaseModel):
    memories: list[Memory] = Field(description="A list of memories about the user.")

In [ ]:
from trustcall import create_extractor
from langchain_openai import ChatOpenAI

## I will inspect the tools made by TrustCall module here...

